# Calling Necessary Libraries AND Getting Today's Date

In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
from scipy.stats import poisson
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Get today's date
given_date = "2025-03-14" #year-month-day
threshold = 70 #threshold for percentages to highlight in final dataframe (between 1 and 100)

wanted_leagues = ['argentina','austria', 'australia', 'belgium', 'brazil', 'colombia','denmark', 
                  'england', 'england2', 'england3', 'england4', 'england5', 'france', 'france2',
                  'germany', 'germany2', 'greece', 'italy', 'italy2',
                   'mexico2', 'netherlands', 'norway','poland', 'portugal', 'portugal2',
                   'saudiarabia','scotland', 'spain', 'spain2', 'sweden', 'switzerland', 'turkey']
# urug, peru, ecuado, usa

In [3]:
#Calculating Days between given date and today
# Today's date
today = datetime.now().date()

# Specific date
specific_date = datetime.strptime(given_date, "%Y-%m-%d").date()

# Calculate the difference in days
difference = specific_date - today

# Add one day to the difference
days_until_specific_date = difference.days + 1

# Scraping Today's Matches and Leagues

In [4]:
URL = "https://www.soccerstats.com/matches.asp?matchday=" + str(days_until_specific_date) + "&listing=2"
page = requests.get(URL)
liqa = []
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="btable")
sth = results.find_all("tr", attrs={'height': '34'})

# Converting Date and Collecting Leagues for Analysis

In [5]:
from dateutil import parser

day_abbreviations = {0: "Mo", 1: "Tu", 2: "We", 3: "Th", 4: "Fr", 5: "Sa", 6: "Su"}
given_date_parsed = parser.parse(given_date)

# Manually format the day to remove leading zeros
day = given_date_parsed.day
month = given_date_parsed.strftime('%b')

# Format the date as "Su 1 Oct" or "Tu 10 Oct" without leading zero for single-digit days
formatted_date = f"{day_abbreviations[given_date_parsed.weekday()]} {day} {month}"
print(formatted_date)

Fr 14 Mar


# Scraping the Web for the League Statistics

In [6]:
final =  pd.DataFrame()
liqa = ''
unique_leagues = wanted_leagues
next_matches = pd.DataFrame()

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    next_matches = pd.concat([next_matches, next_df], ignore_index = True)
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']

# Define current year
current_year = datetime.now().year

# Function to parse and assign the correct year
def parse_date(date_str):
    # Extract day and month
    date_obj = datetime.strptime(date_str[3:], '%d %b')
    # Assign correct year
    full_date = date_obj.replace(year=current_year)
    
    # If the parsed date is already in the future, assign the previous year
    if full_date > datetime.now():
        full_date = full_date.replace(year=current_year - 1)
    
    return full_date

# Apply the function to parse dates
final['Date'] = final['Date'].apply(parse_date)

# Find the latest date for each league
latest_dates = final.groupby('League')['Date'].max().rename('latest_date')

# Merge with the original DataFrame
final = final.merge(latest_dates, on='League')

# Calculate the time difference in days
final['time_diff'] = (final['latest_date'] - final['Date']).dt.days
combined_df = pd.concat([final.head(), final.tail()])
combined_df

,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,latest_date,time_diff
0,2025-01-23,Argentina,Godoy Cruz,Rosario Central,0 - 3,(0-2),0.0,3.0,3.0,0.0,2.0,2.0,2025-03-11,47
1,2025-01-23,Argentina,Tigre,Velez Sarsfield,3 - 0,(2-0),3.0,0.0,3.0,2.0,0.0,2.0,2025-03-11,47
2,2025-01-24,Argentina,Defensa y J.,Banfield,0 - 1,(0-0),0.0,1.0,1.0,0.0,0.0,0.0,2025-03-11,46
3,2025-01-24,Argentina,Lanus,D. Riestra,0 - 2,(0-0),0.0,2.0,2.0,0.0,0.0,0.0,2025-03-11,46
4,2025-01-24,Argentina,Newells,I. Rivadavia,0 - 1,(0-1),0.0,1.0,1.0,0.0,1.0,1.0,2025-03-11,46
7247,2025-03-09,Turkey,Sivasspor,Goztepe,3 - 1,(0-1),3.0,1.0,4.0,0.0,1.0,1.0,2025-03-10,1
7248,2025-03-09,Turkey,Eyupspor,Konyaspor,2 - 1,(1-0),2.0,1.0,3.0,1.0,0.0,1.0,2025-03-10,1
7249,2025-03-09,Turkey,Samsunspor,Adana Demirspor,3 - 2,(2-1),3.0,2.0,5.0,2.0,1.0,3.0,2025-03-10,1
7250,2025-03-09,Turkey,Alanyaspor,Galatasaray,1 - 2,(1-0),1.0,2.0,3.0,1.0,0.0,1.0,2025-03-10,1
7251,2025-03-10,Turkey,Besiktas,Gaziantep,1 - 2,(1-0),1.0,2.0,3.0,1.0,0.0,1.0,2025-03-10,0


In [7]:
next_leagues = next_matches['League'].unique().tolist()
pd.concat([next_matches.head(), next_matches.tail()])

,Date,League,Home,Away,FT,HT
0,Fr 14 Mar,Argentina,Argentinos Jrs,Aldosivi,22:00,
1,Fr 14 Mar,Australia,Adelaide Utd,Macarthur FC,08:35,
2,Fr 14 Mar,Colombia,A. Bucaramanga,D. Pereira,23:30,
3,Fr 14 Mar,England2,Bristol City,Norwich City,20:00,
4,Fr 14 Mar,France,Nice,Auxerre,19:45,
22,Fr 14 Mar,Saudiarabia,Al Nassr,Al Kholood,19:00,
23,Fr 14 Mar,Saudiarabia,Damak,Al Quadisiya,19:00,
24,Fr 14 Mar,Spain,Las Palmas,Alaves,20:00,
25,Fr 14 Mar,Spain2,Real Oviedo,Elche,19:30,
26,Fr 14 Mar,Turkey,Galatasaray,Antalyaspor,17:30,


# Calculating Functions Needed for Dixon-Coles Model

In [8]:
from scipy.optimize import minimize
from scipy.stats import poisson

def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

def solve_parameters_decay(dataset, half_or_full = 'full', xi=0.001, debug = False, init_vals=None, 
                           options={'disp': True, 'maxiter':100},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['Home'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['Away'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Home Teams Not Equal to Away Teams")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0,1.0]) # rho (score correction), gamma (home advantage)
                                     ))
        
    def dc_log_like_decay(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma, t, xi=xi):
        lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
        return  np.exp(-xi*t) * (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
                                  np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        rho, gamma = params[-2:]
        if half_or_full == 'full':
            log_like = [dc_log_like_decay(row.FTHG, row.FTAG, score_coefs[row.Home], defend_coefs[row.Home],
                                      score_coefs[row.Away], defend_coefs[row.Away], 
                                      rho, gamma, row.time_diff, xi=xi) for row in dataset.itertuples()]
        elif half_or_full == 'half':
            log_like = [dc_log_like_decay(row.HTHG, row.HTAG, score_coefs[row.Home], defend_coefs[row.Home],
                                      score_coefs[row.Away], defend_coefs[row.Away], 
                                      rho, gamma, row.time_diff, xi=xi) for row in dataset.itertuples()]
        return -sum(log_like)
    opt_output = minimize(estimate_paramters, init_vals, options=options, constraints = constraints)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))

# Calculating Lambda Values for Dixon-Coles Model

In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
stats_df = pd.DataFrame()
full_time_models = []
half_time_models = []

for league in next_leagues:
    league_df = final[final['League'] == league.capitalize()]
    
    full_time_estimates = solve_parameters_decay(league_df, half_or_full = 'full')
    full_time_models.append(full_time_estimates)

    half_time_estimates = solve_parameters_decay(league_df, half_or_full = 'half')
    half_time_models.append(half_time_estimates)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 289.71794567469846
            Iterations: 86
            Function evaluations: 5558
            Gradient evaluations: 86
Optimization terminated successfully    (Exit mode 0)
            Current function value: 172.01637850642484
            Iterations: 70
            Function evaluations: 4448
            Gradient evaluations: 70
Optimization terminated successfully    (Exit mode 0)
            Current function value: 345.2986769046979
            Iterations: 38
            Function evaluations: 1146
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 241.83357909418518
            Iterations: 34
            Function evaluations: 1020
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 175.04309375884407
            Iterations: 32
            Funct

In [10]:
full_time_models[-1]

{'attack_Adana Demirspor': 0.7449631911281799,
 'attack_Alanyaspor': 0.8963829307886342,
 'attack_Antalyaspor': 0.8923864533187196,
 'attack_Basaksehir': 1.2210888978603294,
 'attack_Besiktas': 1.1799392795603578,
 'attack_Bodrumspor': 0.3167729520972397,
 'attack_Eyupspor': 1.1579534715535322,
 'attack_Fenerbahce': 1.6255000171733782,
 'attack_Galatasaray': 1.609319399699315,
 'attack_Gaziantep': 1.1240023000751893,
 'attack_Goztepe': 1.2726278034968235,
 'attack_Hatayspor': 0.7935001437157151,
 'attack_Kasimpasa': 1.3457499167106202,
 'attack_Kayserispor': 0.8805010542536317,
 'attack_Konyaspor': 0.984035770317893,
 'attack_Rizespor': 1.0198342586738287,
 'attack_Samsunspor': 1.249748049683417,
 'attack_Sivasspor': 0.9492538283269201,
 'attack_Trabzonspor': 1.220466657381054,
 'defence_Adana Demirspor': -0.48402637581477953,
 'defence_Alanyaspor': -0.946996612872874,
 'defence_Antalyaspor': -0.7212132435285877,
 'defence_Basaksehir': -0.9174121281815227,
 'defence_Besiktas': -1.26167

# Calculating Probability Matrices for Half/Full Time

In [11]:
#First Function needs work to make it more understandable and a df rather than matrix!
def dixon_coles_simulate_match(params_dict, homeTeam, awayTeam, max_goals=10):
    team_avgs = [np.exp(params_dict['attack_'+homeTeam] + params_dict['defence_'+awayTeam] + params_dict['home_adv']),
                 np.exp(params_dict['defence_'+homeTeam] + params_dict['attack_'+awayTeam])]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in team_avgs]
    output_matrix = np.outer(np.array(team_pred[0]), np.array(team_pred[1]))
    correction_matrix = np.array([[rho_correction(home_goals, away_goals, team_avgs[0],
                                                   team_avgs[1], params_dict['rho']) for away_goals in range(2)]
                                   for home_goals in range(2)])
    output_matrix[:2,:2] = output_matrix[:2,:2] * correction_matrix
    return output_matrix

full_time_matrices = []
half_time_matrices = []

for i in range(len(next_matches)):
    my_league = next_matches['League'].iloc[i]
    league_index = next_leagues.index(my_league)
    ft_match_score_matrix = dixon_coles_simulate_match(full_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 8)
    ht_match_score_matrix = dixon_coles_simulate_match(half_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 4)
    full_time_matrices.append(ft_match_score_matrix)
    half_time_matrices.append(ht_match_score_matrix)

full_time_matrices[0]

array([[ 1.73182287e-02, -1.05117007e-03,  2.23268228e-04,
         1.33941719e-05,  6.02651265e-07,  2.16923331e-08,
         6.50676632e-10,  1.67292845e-11,  3.76354900e-13],
       [ 5.30470460e-02,  1.37151025e-02,  9.16322177e-04,
         5.49714432e-05,  2.47336006e-06,  8.90281882e-08,
         2.67046248e-09,  6.86591839e-11,  1.54461001e-12],
       [ 1.16104506e-01,  2.08958130e-02,  1.88035337e-03,
         1.12805017e-04,  5.07549751e-06,  1.82691697e-07,
         5.47996464e-09,  1.40893161e-10,  3.16964133e-12],
       [ 1.58836055e-01,  2.85863883e-02,  2.57240585e-03,
         1.54322209e-04,  6.94350312e-06,  2.49930252e-07,
         7.49683190e-09,  1.92748021e-10,  4.33620831e-12],
       [ 1.62971016e-01,  2.93305744e-02,  2.63937300e-03,
         1.58339661e-04,  7.12426253e-06,  2.56436657e-07,
         7.69199605e-09,  1.97765808e-10,  4.44909232e-12],
       [ 1.33770897e-01,  2.40753071e-02,  2.16646679e-03,
         1.29969359e-04,  5.84778210e-06,  2.104899

# Calculating Probabilities of Dixon-Coles Model

In [12]:
ft1, ftx, ft2, ft_score = [], [], [], []
over_15, over_25, under_35, under_45, btts = [], [], [], [], []
ht1, htx, ht2, ht_score, ht_over05, ht_under15 = [], [], [], [], [], []
ho05, ao05, ho15, ao15, hu25, au25 = [], [], [], [], [], []

# Helper function to calculate total goals for each score
def total_goals(i, j):
    return i + j

for i in range(len(next_matches)):
    my_matrix = full_time_matrices[i]
    ht_matrix = half_time_matrices[i]

    ft1.append(round(np.sum(np.tril(my_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    ftx.append(round(np.sum(np.diag(my_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ft2.append(round(np.sum(np.triu(my_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)
    
    max_score = np.unravel_index(np.argmax(my_matrix), my_matrix.shape) # Find the index of the maximum score
    home_goals, away_goals = max_score
    ft_score.append(f"{home_goals}-{away_goals}") # Format the score as 'home-away'

    # Calculate the probabilities
    over_15.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 1.5]) * 100, 2))
    over_25.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 2.5]) * 100, 2))
    under_35.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 3.5]) * 100, 2))
    under_45.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 4.5]) * 100, 2))

    # Calculate BTTS (both teams to score and goals != 0)
    btts.append(round(np.sum([my_matrix[i, j] for i in range(1, my_matrix.shape[0]) for j in range(1, my_matrix.shape[1])]) * 100, 2)) 

    # Calculate statistics for Half Time
    ht1.append(round(np.sum(np.tril(ht_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    htx.append(round(np.sum(np.diag(ht_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ht2.append(round(np.sum(np.triu(ht_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)

    ht_max_score = np.unravel_index(np.argmax(ht_matrix), ht_matrix.shape) # Find the index of the maximum score
    ht_hogs, ht_awgs = ht_max_score
    ht_score.append(f"{ht_hogs}-{ht_awgs}") # Format the score as 'home-away'

    ht_over05.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) > 0.5]) * 100, 2))   
    ht_under15.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) < 1.5]) * 100, 2)) 

    ho05.append(round(np.sum(my_matrix[1:,:]) * 100, 2))
    ao05.append(round(np.sum(my_matrix[:,1:]) * 100, 2))
    ho15.append(round(np.sum(my_matrix[2:,:]) * 100, 2))
    ao15.append(round(np.sum(my_matrix[:,2:]) * 100, 2))
    hu25.append(round(np.sum(my_matrix[:3,:]) * 100, 2))
    au25.append(round(np.sum(my_matrix[:,:3]) * 100, 2))
    

# Combine lists into a DataFrame
final_results = pd.DataFrame({
    'League': next_matches['League'], 'Home': next_matches['Home'], 'Away': next_matches['Away'],
    'FT1': ft1, 'FTX': ftx, 'FT2': ft2, 'FTR': ft_score,
    'DC1X': [x + y for x, y in zip(ft1, ftx)], 'DC12': [x + y for x, y in zip(ft1, ft2)], 'DCX2': [x + y for x, y in zip(ftx, ft2)],
    '1.5O': over_15, '2.5O': over_25, '3.5U': under_35, '4.5U': under_45, 'BTTS': btts,
    'HT1': ht1, 'HTX': htx, 'HT2': ht2, 'HTR': ht_score,
    'HTDC1X': [x + y for x, y in zip(ht1, htx)], 'HTDC12': [x + y for x, y in zip(ht1, ht2)], 'HTDCX2': [x + y for x, y in zip(htx, ht2)],
    'HT0.5O': ht_over05, 'HT1.5U': ht_under15, 'H0.5O':ho05, 'A0.5O':ao05, 'H1.5O':ho15, 'A1.5O':ao15, 'H2.5U':hu25, 'A2.5U':au25
})

# Function to highlight values higher than threshold
def highlight_values(value):
    if isinstance(value, str):
        return ''  # Return empty string for NaN values
    elif value > threshold:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = final_results.style.applymap(highlight_values)
styled_df.to_excel(given_date + ".xlsx", index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

,League,Home,Away,FT1,FTX,FT2,FTR,DC1X,DC12,DCX2,1.5O,2.5O,3.5U,4.5U,BTTS,HT1,HTX,HT2,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O,HT1.5U,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U
0,Argentina,Argentinos Jrs,Aldosivi,94.200000,3.310000,0.030000,4-0,97.510000,94.230000,3.340000,90.610000,77.600000,38.000000,57.350000,16.150000,78.030000,17.870000,0.800000,1-0,95.900000,78.830000,18.670000,80.120000,46.460000,95.890000,16.070000,89.110000,1.400000,22.320000,97.450000
1,Australia,Adelaide Utd,Macarthur FC,40.010000,20.920000,39.030000,1-1,60.930000,79.040000,59.950000,90.200000,74.960000,44.910000,64.430000,73.780000,28.140000,37.140000,34.460000,0-0,65.280000,62.600000,71.600000,77.930000,53.490000,86.060000,85.700000,58.630000,57.940000,68.390000,69.070000
2,Colombia,A. Bucaramanga,D. Pereira,19.110000,46.820000,34.070000,0-0,65.930000,53.180000,80.890000,28.040000,8.500000,97.940000,99.590000,16.020000,11.640000,52.900000,35.420000,0-0,64.540000,47.060000,88.320000,55.940000,76.540000,32.990000,46.470000,6.160000,13.010000,99.210000,97.440000
3,England2,Bristol City,Norwich City,48.770000,25.280000,25.940000,1-1,74.050000,74.710000,51.220000,78.050000,53.920000,68.380000,84.190000,56.390000,35.270000,44.280000,20.400000,0-0,79.550000,55.670000,64.680000,64.850000,72.800000,81.250000,68.720000,49.870000,32.370000,76.400000,88.750000
4,France,Nice,Auxerre,59.560000,22.310000,18.090000,1-1,81.870000,77.650000,40.400000,85.580000,64.590000,57.400000,75.740000,61.720000,44.890000,31.170000,23.370000,1-0,76.060000,68.260000,54.540000,82.820000,48.050000,88.630000,68.320000,63.960000,31.930000,62.860000,88.960000
5,France2,AC Ajaccio,Red Star,48.680000,22.150000,29.170000,1-0,70.830000,77.850000,51.320000,62.060000,38.340000,81.480000,92.500000,40.710000,26.670000,43.330000,29.950000,0-0,70.000000,56.620000,73.280000,65.680000,73.290000,72.950000,59.970000,37.580000,23.320000,85.540000,93.460000
6,France2,Amiens,Caen,50.660000,21.270000,28.070000,1-0,71.930000,78.730000,49.340000,64.560000,41.110000,79.380000,91.300000,42.620000,38.560000,40.930000,20.420000,0-0,79.490000,58.980000,61.350000,68.060000,70.690000,75.230000,60.610000,40.660000,23.910000,83.460000,93.180000
7,France2,Annecy,Rodez Aveyron,45.010000,17.730000,37.230000,2-1,62.740000,82.240000,54.960000,85.120000,69.350000,51.950000,71.030000,67.140000,37.330000,32.220000,30.160000,1-0,69.550000,67.490000,62.380000,80.990000,54.090000,84.810000,81.600000,56.210000,50.470000,70.740000,75.870000
8,France2,Clermont,Grenoble,36.050000,24.160000,39.790000,0-1,60.210000,75.840000,63.950000,57.220000,33.320000,85.050000,94.400000,37.590000,19.860000,48.310000,31.800000,0-0,68.170000,51.660000,80.110000,58.660000,79.510000,62.560000,65.150000,25.780000,28.420000,92.290000,90.950000
9,France2,Martigues,Pau FC,32.610000,23.430000,43.960000,0-1,56.040000,76.570000,67.390000,58.990000,35.120000,83.800000,93.750000,38.770000,26.390000,49.370000,24.230000,0-0,75.760000,50.620000,73.600000,57.650000,80.390000,60.970000,68.730000,24.240000,32.380000,93.030000,88.750000
